# 2. Load Measurement Data for Our Target Site

Now that we’ve identified our target station (BURNABY SOUTH, NAPS ID: 100119), let’s load the actual measurement data collected at this site.

We will use data from **2020 to 2023**.

The cell below imports the required packages.


In [ ]:
import zipfile
import os
import pandas as pd
import sys
from pathlib import Path
from pprint import pprint

# set project root
sys.path.insert(0, str(Path.cwd().parent))

from src.config import *
from src.data_cleaning import *
from src.download_data import *
from src.load_data import *

In [ ]:
# BURNABY SOUTH
site_id = 100119

years = range(2020, 2024)


## 2.1. Download the Data

The code below will download the NAPS integrated measurement data from **2020 to 2023**. These files include air pollutant data collected at the **BURNABY SOUTH** station.

<details>
  <summary><strong>Optional: Manual Download (click to expand)</strong></summary>

  If the automatic download fails, you can download the files manually:

  1. Visit the [NAPS data portal](https://data-donnees.az.ec.gc.ca/data/air/monitor/national-air-pollution-surveillance-naps-program/)
  2. Navigate to:  
     `Data-Donnees/` → `2020/` → `IntegratedData-DonneesPonctuelles/` →  
     `2020_IntegratedPM2.5-PM2.5Ponctuelles.zip`
  3. Save the file to: `../data/raw/integrated/`
  4. Repeat steps 2–3 for the years **2021, 2022, and 2023**.
</details>


In [ ]:
download_integrated_pm25()

## 2.2. Unzip the Downloaded Files

After downloading the data, we need to unzip the files for each year from **2020 to 2023**.

The cell below will extract all ZIP files to the `../data/raw/integrated_data/` folder.


In [ ]:
# create a list from 2020 (inclusive) to 2024 (exclusive)
years = range(2020, 2024)

for year in years: 
    zip_path = f'../data/raw/integrated/{year}_IntegratedPM2.5-PM2.5Ponctuelles.zip'
    extract_to = '../data/raw/integrated_data'

    if os.path.exists(zip_path):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Extracted: {zip_path}")
    else:
        print(f"File not found: {zip_path}")

## 2.3. Check the File and Data

Before loading the data into Python, let’s take a moment to open one of the unzipped files manually and see what it contains.

You can use any spreadsheet program to open the file. I recommend starting with the **2020** file:

`../data/raw/integrated/2020_IntegratedPM2.5-PM2.5Ponctuelles/S100119_PM25_2020_EN.xlsx`

### What You’ll See and Explore

The file contains **integrated PM2.5 speciation data**, organized into multiple worksheets by measurement type.

- The `Station Info` sheet includes metadata such as `Sampling Frequency` for each site and year.
- We will focus on the `Metals_ICPMS (Near-Total)` sheet, which contains time series data for various metal species.
- Use the `Sampling Type` column to distinguish routine measurements from field blanks.
- Missing values are common in some rows and should be expected.

As you explore the data:

- Check which parameters were measured and in what units
- Note how frequently data was collected (e.g., every 3 or 6 days)
- Look for patterns, missing values, or anomalies

> **Try this:**
> 
> Open the **2021** file as well and compare it with 2020.
> 
> `../data/raw/integrated/2021_IntegratedPM2.5-PM2.5Ponctuelles/S100119_PM25_2021_EN.xlsx`
>
> You’ll notice the set of worksheets isn’t identical — Which sheets are new or missing? 

Once you're familiar with the structure, we’ll load the files into Python in the next step.


## 2.4. Load and Preview the Measurement Data

The code below loads the measurement data from the **Metals_ICPMS (Near-Total)** worksheet (and others, if needed) for each year from 2020 to 2023.

Each Excel file contains multiple worksheets, and the structure may vary slightly from year to year. The function `get_sheets_for_year(year)` helps map consistent sheet labels (e.g., `'nt'`) to the correct worksheet name for each year.

For each year:

- We load the data from the selected sheets (e.g., `'Metals_ICPMS (Near-Total)'`)
- Skip the first 9 rows (which contain metadata)
- Use the 10th row as the header (column names)
- Display the first 3 rows of each loaded sheet to give you a preview

This helps confirm that the file paths, sheet names, and data structure are correct before we proceed to combine and analyze the data.


In [ ]:
for year in years:
    print(f"\n--- Year {year} ---")
    
    sheet_map = get_sheets_for_year(year)
    # shows which sheets exist this year
    pprint(sheet_map)

    for key in sheet_map:  # e.g. 'nt', 'ws', ...
        df = load_target_sheet(year, site_id, key)
        print(f"\n▶ {key} · {len(df):,} rows")
        display(df.head(3))


## 2.5. Clean and Save the Data

Before saving the measurement data, we’ll simplify each sheet:

- Drop columns ending in `-VFlag`, which contain validation flags that are not useful for our analysis
- Rename key columns (e.g., `Sampling Date` → `sampling_date`)
- Shorten analyte column names by keeping only the abbreviation in parentheses (e.g., `Selenium (Se)` → `Se`)

The cleaned data will be saved as CSV files in the `data/processed/` folder, one file per year and data type.


In [ ]:
for year in years:
    print(f"\nSaving cleaned data for {year}...")

    sheet_map = get_sheets_for_year(year)

    for key in sheet_map:
        df = load_target_sheet(year, site_id, key)

        df_clean = (
            df.pipe(drop_vflag_cols)
            .pipe(rename_cols)
            .pipe(normalize_analyte_names)
        )

        df_clean = df_clean.set_index('sampling_date')

        out_path = PROCESSED_DATA_DIR / f"{year}_{site_id}_{key}.csv"
        out_path.parent.mkdir(parents=True, exist_ok=True)

        df_clean.to_csv(
            out_path,
            index_label="sampling_date",
            date_format="%Y-%m-%d"
        )

        print(f"Saved: {out_path.name}")



The cell below will save PM2.5 data.

In [ ]:
for year in years:
    print(f"\nSaving cleaned PM2.5 data for {year}...")
    
    pm25_df = load_target_sheet(year, site_id, 'pm25')
    
    # Keep only first PM2.5 and MDL
    pm25_subset = pm25_df[[
        'NAPS Site ID', 'Sampling Date', 'Sample Type',
        'PM2.5', 'PM2.5-MDL'
    ]].copy()
    

    pm25_subset = convert_micro_to_nano(pm25_subset)
    
    df_clean = (
        pm25_subset.pipe(rename_cols)
                   .pipe(normalize_analyte_names)
                   .rename(columns={'PM2.5': 'PM25', 'PM2.5-MDL': 'PM25-MDL'})
                   .set_index("sampling_date")
    )

    out_path = PROCESSED_DATA_DIR / f"{year}_{site_id}_pm25.csv"
    out_path.parent.mkdir(parents=True, exist_ok=True)
    
    df_clean.to_csv(
            out_path,
            index_label="sampling_date",
            date_format="%Y-%m-%d"
    )

    display(df_clean.head(3))